In [1]:
from datasets import load_from_disk

ds = load_from_disk('ds-for-synth-gen')

In [2]:
ds[0]

{'tibetan': 'གཞན་ལ་བགོ་སྐལ་བྱེད་དུ་མ་མཆིས་ཤིང་།།',
 'phonetic': 'zhen la go kal jé duma chi shing',
 'english': 'Will not be apportioned to anybody else.'}

In [21]:
from transformers import PreTrainedTokenizerFast, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained('t5-base/checkpoint-16875', device_map='cuda:0')

# Initialize and train the tokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained('custom-tokenizer', device='cuda:0')

In [24]:
input_ids = tokenizer("translate Tibetan to English: "+ds[0]['tibetan'], return_tensors="pt").to('cuda:0').input_ids
outputs = model.generate(input_ids)
translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
translation

'And the great wisdom of the three times of the three times'

In [25]:
def add_synth_data(example):
    input_ids = tokenizer("translate Tibetan to English: "+example['tibetan'], return_tensors="pt").to('cuda:0').input_ids
    outputs = model.generate(input_ids)
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    example['synthetic'] = translation
    return example

In [32]:
ds = ds.map(add_synth_data)

Map:   0%|          | 0/22500 [00:00<?, ? examples/s]

In [33]:
ds[0]

{'tibetan': 'གཞན་ལ་བགོ་སྐལ་བྱེད་དུ་མ་མཆིས་ཤིང་།།',
 'phonetic': 'zhen la go kal jé duma chi shing',
 'english': 'Will not be apportioned to anybody else.',
 'synthetic': 'And the great wisdom of the three times of the three times'}

In [34]:
ds.save_to_disk('ds-with-synth-data')

Saving the dataset (0/1 shards):   0%|          | 0/22500 [00:00<?, ? examples/s]